# Mentoría 'de cómo clasificar en géneros a las canciones'
## Práctico III : Introducción al aprendizaje automático

**Antes de empezar:**
- [Instalar spaCy y el modelo de lenguaje con el que van a trabajar](https://spacy.io/models#quickstart)

**Consideraciones:**
- Se evalúa el estilo y prolijidad del código.
- Se permite hacer trabajo extra, siempre y cuando las actividades básicas estén resueltas.

**Recomendación:**
- Hay muchos ejemplos de código en internet, no se sientan obligados a implementar todo desde cero.

### Librerías

In [ ]:
!pip3 install spotipy
!pip3 install pandas
!pip3 install spacy
!pip3 install pymusixmatch
!pip3 install nltk
!pip3 install sklearn

# Agregar las librerías extra que se utilicen en esta celda y la siguiente

### Dependencias y acceso a APIs

In [178]:
import pandas as pd
import numpy as np
import spotipy
import spacy
from spotipy.oauth2 import SpotifyClientCredentials
from musixmatch import Musixmatch
import seaborn as sns
import tqdm
import plotly.express as px
import plotly.graph_objects as go

client_id = '46b333d567314a89a6254b6c6b054be6'
client_secret = '9d922c3613e441518349dcf55f7d5853'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# es = es_core_web_sm
nlp = spacy.load("es") # completar con el modelo que van a utilizar

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
musixmatch = Musixmatch('1aa5272f4402bf2f082ad2f3958c2c62') # se puede reemplazar por otra API si da mejores resultados

In [4]:
sns.set_context(context='paper')

### 1) Recopilar los datos obtenidos en los prácticos anteriores

Para esta parte consideraremos [la playlist colaborativa de la mentoría](https://open.spotify.com/playlist/2IuD0qZb14cji5y52crdsO?si=nfHRPDquQRyotEcXc4tG7Q), de esta obtendremos:
- Las features del audio de las canciones
- Las features textuales de sus letras

Además es necesario aplicar el mismo preprocesamiento que aplicamos en los prácticos anteriores para ambos tipos de features (el preprocesamiento del p1 a las features de audio y el de p2 al de features textuales) y obtener el género de cada canción, que en caso de ser más de uno para una canción el equipo deberá discutir una estrategia para estos casos y comentarla en el informe.

Luego, se separará al dataset resultante en **X** e **y**, donde:
- X es el conjunto de features
- y es la etiqueta, en este caso el género de la canción, que deberá ser codificado en valores del tipo **int**

Por último, se dividirá a estos dos conjuntos en los splits **train** y **test**

**Recomendaciones:**
- Obtener las features por separado y hacer un join de los datasets.
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)
- Si usan features categóricas, ENCODEARLAS!

# Dataset

## Music Features

In [186]:
#Aux funcs
def genres_by_artist_id(id): #id: str
    artist = sp.artist(id)
    genres = artist['genres']
    return genres    #genres: List[str]

def songs_from_album_id(album_id):
    songs = []
    album = sp.album(album_id)
    artist = album['artists'][0]['name']
    for item in album['tracks']['items']:
        track = {}
        track["song_name"] = item['name']
        track["song_id"] = item['id']
        track["album_name"] = album['name']
        track["album_id"] = album["id"]
        audio_features = sp.audio_features(track["song_id"])
        track["audio_features"] = audio_features[0]
        track["artist"] = artist
        songs.append(track)
    return songs    #songs:List[dict]

def get_genres(artists_id):
    res = [genres_by_artist_id(x) for x in artists_id if genres_by_artist_id(x)!= []]
    if res != []:
        return res[0]
    return res

def add_track(track_id, songs): #track_id:str, songs:List[dict]
    track = sp.track(track_id)
    audio_features = sp.audio_features(track_id)
    row = {}
    row["song_name"] = track['name']
    row["song_id"] = track['id']
    row["artists"] = [x["name"] for x in track["artists"]]
    row["artists_id"] = [x["id"] for x in track["artists"]]
    row["album_name"] = track['album']['name']
    row["album_id"] = track['album']['id']
    row["audio_features"] = audio_features[0]
    row["genres"] = get_genres(row["artists_id"])
    songs.append(row)
    return songs


In [187]:
PLAYLIST_ID = "2IuD0qZb14cji5y52crdsO"
songs = []
def add_songs_of_playlist(playlist_id,songs_array):
    print("This may take a while...15 min aprox")
    offset = 0
    playlist = sp.playlist_tracks(playlist_id)
    for i in tqdm.tqdm(range(playlist["total"])):
        j = i%100
        if i == 99:
            offset += 1
            playlist = sp.playlist_tracks(playlist_id, offset=i+offset,limit=100)
        add_track(playlist["items"][j]["track"]["id"],songs_array)
    return


add_songs_of_playlist(PLAYLIST_ID,songs)


This may take a while...


100%|██████████| 1190/1190 [22:05<00:00,  1.11s/it]


In [193]:
songs_original_df = pd.DataFrame(songs)
songs_original_df.sample(4)

,album_id,album_name,artists,artists_id,audio_features,genres,song_id,song_name
754,09LIOeWRW3Hh5y6RrhNM7q,Homenaje (Los Amigos del Mismo Palo),"[La Mona Jimenez, Pedrito Gazzoni]","[64DFKvGarD5nmkfaIiiakf, 0cMYVdNOG3QoAPs5Al3zT4]","{'danceability': 0.365, 'energy': 0.887, 'key'...",[cuarteto],3FKa6b0cRdZOJFKw4G7pjK,Amor de compra y venta
608,50PvQ5VYnLUuXd1DZIJBhN,Cuarteto es La Mona,"[La Mona Jimenez, Eduardo Pupi Rojo and Fredy ...","[64DFKvGarD5nmkfaIiiakf, 66jFkBAVURODl7IqekuCon]","{'danceability': 0.645, 'energy': 0.87, 'key':...",[cuarteto],537bIbWL67J8tX9Yg3sLTo,Si yo fuera mujer
474,0C8MDwGT2rGDPLVR97phtn,La Mona y El Hombre,"[La Mona Jimenez, Sergio Oliva and Fredy Zabaley]","[64DFKvGarD5nmkfaIiiakf, 1gKMLNoW1i6ZpeBSawjxKE]","{'danceability': 0.655, 'energy': 0.629, 'key'...",[cuarteto],47YOWtUVpWFv6ipAHJasNH,El ruego de la nena
255,0ZfDwnnPHU8GHUZ7GhVcY2,La Magia de La Mona (En Vivo '92),[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.673, 'energy': 0.641, 'key'...",[cuarteto],7ErQosMEgLklgNS30Oin4j,La Pupera - En Vivo


Checking dataframe's consistency.

In [225]:
songs_original_df[[x == [] for x in songs_original_df["genres"]]]

,album_id,album_name,artists,artists_id,audio_features,genres,song_id,song_name
48,59tn7tvd1M5XNWwV3TaVWC,"Enrique Santos Discepolo ""El poeta del tango"" ...",[Enrique Santos Discépolo],[0aPYs7yoiP2NtS5xNZXKjg],"{'danceability': 0.492, 'energy': 0.541, 'key'...",[],3PI0FE7JUmEmEyN5YgKPZA,Cambalache


Only one cell doesn't have a genre. This is an edge case; Cambalache's genre is empty. I decided to assign to this the same genres Julio Sosa has.

In [230]:
songs_original_df.iloc[48]["genres"] = genres_by_artist_id("7Cg2eqV6oHNE0P54WfajIX")

No nulls in any other column

In [239]:
columns = songs_original_df.columns

for column in columns:
    if songs_original_df[[x == [] for x in songs_original_df[column]]].empty:
        print("No nulls in column ", column)

No nulls in column  album_id
No nulls in column  album_name
No nulls in column  artists
No nulls in column  artists_id
No nulls in column  audio_features
No nulls in column  genres
No nulls in column  song_id
No nulls in column  song_name


In [263]:
#duplicated_songs = songs_original_df[songs_original_df["song_id"].duplicated(keep='last')]
#duplicated_songs.count()
songs_original_df.duplicated(subset=["song_id"])
songs_original_df.iloc[1161]
songs_original_df[songs_original_df["song_id"]=="7fXqqODhAd4Qq4Pa3lI1XA"]

,album_id,album_name,artists,artists_id,audio_features,genres,song_id,song_name
161,142WuAv2UZ0PmJq4eVlMq0,"La Mona en Vivo, Vol. 1",[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.363, 'energy': 0.913, 'key'...",[cuarteto],7fXqqODhAd4Qq4Pa3lI1XA,Gira el mundo al revés - En Vivo
261,142WuAv2UZ0PmJq4eVlMq0,"La Mona en Vivo, Vol. 1",[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.363, 'energy': 0.913, 'key'...",[cuarteto],7fXqqODhAd4Qq4Pa3lI1XA,Gira el mundo al revés - En Vivo
361,142WuAv2UZ0PmJq4eVlMq0,"La Mona en Vivo, Vol. 1",[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.363, 'energy': 0.913, 'key'...",[cuarteto],7fXqqODhAd4Qq4Pa3lI1XA,Gira el mundo al revés - En Vivo
461,142WuAv2UZ0PmJq4eVlMq0,"La Mona en Vivo, Vol. 1",[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.363, 'energy': 0.913, 'key'...",[cuarteto],7fXqqODhAd4Qq4Pa3lI1XA,Gira el mundo al revés - En Vivo
561,142WuAv2UZ0PmJq4eVlMq0,"La Mona en Vivo, Vol. 1",[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.363, 'energy': 0.913, 'key'...",[cuarteto],7fXqqODhAd4Qq4Pa3lI1XA,Gira el mundo al revés - En Vivo
661,142WuAv2UZ0PmJq4eVlMq0,"La Mona en Vivo, Vol. 1",[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.363, 'energy': 0.913, 'key'...",[cuarteto],7fXqqODhAd4Qq4Pa3lI1XA,Gira el mundo al revés - En Vivo
761,142WuAv2UZ0PmJq4eVlMq0,"La Mona en Vivo, Vol. 1",[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.363, 'energy': 0.913, 'key'...",[cuarteto],7fXqqODhAd4Qq4Pa3lI1XA,Gira el mundo al revés - En Vivo
861,142WuAv2UZ0PmJq4eVlMq0,"La Mona en Vivo, Vol. 1",[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.363, 'energy': 0.913, 'key'...",[cuarteto],7fXqqODhAd4Qq4Pa3lI1XA,Gira el mundo al revés - En Vivo
961,142WuAv2UZ0PmJq4eVlMq0,"La Mona en Vivo, Vol. 1",[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.363, 'energy': 0.913, 'key'...",[cuarteto],7fXqqODhAd4Qq4Pa3lI1XA,Gira el mundo al revés - En Vivo
1061,142WuAv2UZ0PmJq4eVlMq0,"La Mona en Vivo, Vol. 1",[La Mona Jimenez],[64DFKvGarD5nmkfaIiiakf],"{'danceability': 0.363, 'energy': 0.913, 'key'...",[cuarteto],7fXqqODhAd4Qq4Pa3lI1XA,Gira el mundo al revés - En Vivo


In [254]:
no_duplicates = songs_original_df.drop_duplicates(subset=["song_id"], keep='last')
no_duplicates.count()
no_duplicates

,album_id,album_name,artists,artists_id,audio_features,genres,song_id,song_name
0,1MQO4j8QExVgmnplbIodEU,Arca,[Arca],[4SQdUpG4f7UbkJG3cJ2Iyj],"{'danceability': 0.161, 'energy': 0.482, 'key'...","[art pop, dance pop, deconstructed club, elect...",7j9DYPyCuvSAtPcevpAkzb,Desafío
1,1MQO4j8QExVgmnplbIodEU,Arca,[Arca],[4SQdUpG4f7UbkJG3cJ2Iyj],"{'danceability': 0.23, 'energy': 0.434, 'key':...","[art pop, dance pop, deconstructed club, elect...",1cwTMSQeMaA9fVKEF1iWeD,Anoche
2,1MQO4j8QExVgmnplbIodEU,Arca,[Arca],[4SQdUpG4f7UbkJG3cJ2Iyj],"{'danceability': 0.289, 'energy': 0.28, 'key':...","[art pop, dance pop, deconstructed club, elect...",0aL27vskbMpwsMGUkHm3Zf,Sin Rumbo
3,1QXxmsxolhkqiFtI1mpX4i,Sus 16 Grandes Exitos,[Rocío Dúrcal],[2uyweLa0mvPZH6eRzDddeB],"{'danceability': 0.499, 'energy': 0.648, 'key'...","[bolero, cancion melodica, grupera, latin, lat...",2kfSFdq2h0xLXq01em1zc7,La Gata Bajo la Lluvia
4,1xrQ48Vvnvm3SmAbnIukGt,Recuerdos II,[Juan Gabriel],[2MRBDr0crHWE5JwPceFncq],"{'danceability': 0.528, 'energy': 0.383, 'key'...","[cancion melodica, latin, latin pop]",5ySxlyvySBhIEvoO2xx7uT,Querida
5,70Bq990gBdpLDzx7M8r28i,Victimas del Vaciamiento,[Hermetica],[6j6Ld5h0aFgH0VQWQNazS7],"{'danceability': 0.33, 'energy': 0.574, 'key':...","[argentine heavy metal, argentine metal, argen...",4c3ix86CIDIVzXBTRUNwBd,Otro Día para Ser
6,33XPcITW6McI85shkX51RP,Acido Argentino,[Hermetica],[6j6Ld5h0aFgH0VQWQNazS7],"{'danceability': 0.406, 'energy': 0.947, 'key'...","[argentine heavy metal, argentine metal, argen...",6JnnF3ZIpuJt51oG9wVAEb,Robo un Auto
7,5IyVKrEhTZhhTbUsB8xdl6,Mundo Guanaco,[Almafuerte],[6qYd7xlmeeeDkPfx6mZ9PV],"{'danceability': 0.577, 'energy': 0.85, 'key':...","[argentine heavy metal, argentine metal, argen...",6p5SbKcAnIECU3hoFkZlPQ,El Pibe Tigre
8,0zHdtlc987CwJR3Pr5TV5z,Un Paso Mas en la Batalla,[V8],[2GOgGMMJooNV8Yk2PjvzAa],"{'danceability': 0.288, 'energy': 0.982, 'key'...","[argentine heavy metal, argentine metal, latin...",2zumIZU09Iz2eGRJ5UXHms,Deseando Destruir y Matar
9,1KUXCG2YMNtEPZx8MkZ1E9,Serie De Oro,[Malon],[3MBsvBr8B6mfjO6txfT6uL],"{'danceability': 0.467, 'energy': 0.891, 'key'...","[argentine heavy metal, argentine metal, argen...",2lkEfig7F9vg1MALElBd0z,Nido De Almas


In [232]:
audio_features_base = pd.DataFrame(list(songs_original_df["audio_features"].values))
audio_features_description = audio_features_base.describe()

In [233]:
#more aux functions
def track_by_feature(feature, value):
    track_id = audio_features_base[audio_features_base[feature]==value]['id']
    track_id = track_id.values.item(0)
    return songs_original_df[songs_original_df['song_id']== track_id]
#example use: 
#print(track_by_feature("valence",0.039100))
#track_by_feature("speechiness",0.492000)

def songs_of_description(statistic):
    row = audio_features_description.loc[statistic]
    keys = row.keys()
    tracks_of_row = []
    for key in keys:
        track = track_by_feature(key,row[key]).to_dict()['song_name']
        track = list(track.values())[0]
        tracks_of_row.append({key: track})
    return tracks_of_row
#example use
#songs_of_description("min")

songs_of_description("max")

[{'acousticness': 'Que Cruz La Que Lleva El Viento'},
 {'danceability': 'Niña de Tilcara'},
 {'duration_ms': 'Escribele una carta - En Vivo'},
 {'energy': 'No Me Arrepiento de Este Amor - En Vivo'},
 {'instrumentalness': 'En la Ciudad de la Furia'},
 {'key': 'Robo un Auto'},
 {'liveness': 'Amor de mañana'},
 {'loudness': 'Quién Se Tomó Todo el Vino - En Vivo'},
 {'mode': 'Sin Rumbo'},
 {'speechiness': 'Te voy a enseñar - En Vivo'},
 {'tempo': 'Amigos'},
 {'time_signature': 'Desafío'},
 {'valence': 'Ahogadito - En Vivo'}]

## Lyrics

Decisión de diseño
El género es una lista de géneros.

Hay canciones que no tienen un género asociado, en ese caso la api devuelve un []. Cómo tratar ese caso?

Yo propongo usar el género del artista. 

Qué pasa si shakira y metallica colaboran? qué género le ponés? Una decision posible: los géneros de los dos.

distribución de los géneros en el dataset.

### 2) Elegir tres modelos de clasificadores multiclase

Aquí escogeremos tres modelos diferentes y luego compararemos su rendimiento para esta tarea. El procedimiento será el siguiente:
- Inicializar los modelos
- Entrenarlos usando el split **train** de los datos

**Recomendación:**
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)

### 3) Informe: Comparar el rendimiento de los modelos

Una vez entrenados los tres modelos, compararemos su rendimiento:
- Correr los modelos usando el split **test**
- Obtener el reporte de clasificación y la matriz de confusión para cada modelo
- Graficar llevando a 2 dimensiones nuestro split **test** pintando con colores diferentes según la etiqueta correspondiente.
- Graficar de manera similar los resultados obtenidos con cada clasificador y sobre esto la función de clasificación obtenida.
- Guardar los modelos usando **pickle**
- Discutir los resultados obtenidos

**Recomendación:**
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)

### 4) Tareas adicionales:

Estas tareas servirán para extrapolar un poco el trabajo básico, y también sumarán puntos extra. Deben elegir una o más de las siguientes:
-  Análisis sobre el balance de clases del dataset, balanceo usando **subsampling** u **oversampling** y comparación de resultados vs el modelo básico
- Optimización de hiperparámetros y comparación de resultados vs el modelo básico
- Graficar importancia de features
- Graficar correlación de features

**Recomendación:**
- Hacer varias ahora puede ahorrarles tiempo en el futuro